## There are some instructions you need to follow:
<li> You only need to write your code in the comment area "Your Code Here".</li>
<li>Do not upload your own file. Please make the necessary changes in the Jupyter notebook file already present in the server.</li>
<li>Please note, there are several cells in the Assignment Jupyter notebook that are empty and read only. Do not attempt to remove them or   edit them. They are used in grading your notebook. Doing so might lead to 0 points.</li>

In [60]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity

# Question 1

In [61]:
"""
Question 1

Write a function that takes the file name of the Wikipedia page containing all Greek ancient
philosophers (saved as "Index.html" in your workspace) and returns a list tuples containing 
the name of the philosopher and the path to its individual article file.

Example of use: get_philosophers("Index.html")

The output should be a list of tuples:

[('Acrion', 'Philosophers/Acrion.html'),
 ('Adrastus of Aphrodisias', 'Philosophers/Adrastus of Aphrodisias.html'),
 ('Aedesia', 'Philosophers/Aedesia.html'),
 ('Aedesius', 'Philosophers/Aedesius.html'),
 ('Aeneas of Gaza', 'Philosophers/Aeneas of Gaza.html'),
 ('Aenesidemus', 'Philosophers/Aenesidemus.html'),
 ...]
 
  
NOTE: For processing speed purposes, the table in "Index.html" has been shortened compared
to the one online on wikipedia.org. Do not worry if you do not find some philosophers in 
your results, this is made on purpose. 

"""

def get_philosophers(filename):
    
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(filename, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(),'lxml')
    
    philosopher_list = []
    
    tableRows = soup.find_all('table',class_='wikitable sortable')[0]
    data_rows = tableRows.find_all('tr')[1:]
    philosopher_name = [] #list containing each philosopher's name
    # grabs philosopher name from title an appends to list
    for data in data_rows:
        philosopher_name.append(data.find('a').get('title'))

    # adding path and filename at the end of philosopher's name
    for name in philosopher_name:
        x = name
        y = name,"Philosophers/"+ x + '.html'
        philosopher_list.append(y)

    # returns philosopher list as tuple
#     philosopher_tuple_list = tuple(philosopher_list)
    return philosopher_list

# Once done, try this:
filenames = get_philosophers("Index.html")
# filenames

# Question 1: Non-Function

In [62]:
import codecs
from bs4 import BeautifulSoup
f = codecs.open("Index.html", 'r', 'utf-8')
soup = BeautifulSoup(f.read(),'lxml')

# reference: http://savvastjortjoglou.com/nba-draft-part01-scraping.html
# tuples of philosopher name w/ path 
philosopher_list = []

# creates list of philosopher name
tableRows = soup.find_all('table',class_='wikitable sortable')[0]
data_rows = tableRows.find_all('tr')[1:]
philosopher_name = [] #list containing each philosopher's name

# grabs philosopher name from title and appends to list
for data in data_rows:
    philosopher_name.append(data.find('a').get('title'))
#     print(type(data))


# # loops through a list indices 
# for i in range(len(data_rows)):
#     # for each table data element from each table row
#     td = data_rows[i].find_all('td')[0].get_text().strip()
# #     philosopher_name.append(td)


# adding path and filename at the end of philosopher's name
for name in philosopher_name:
    x = name
    y = name,"Philosophers/"+ x + '.html'
    philosopher_list.append(y)

# philosopher_list

# Question 2

In [63]:
"""
Question 2


Write a function that scrapes the text on a philosophers’s page and returns it as a text 
string. The input is the name of the file that contains the philosopher's page.

Example of use: get_text('Philosophers/Acrion.html')
should output the text of the page.
'Acrion was a Locrian and a Pythagorean philosopher...'
"""

def get_text(file):
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(file, 'r', 'utf-8')
    page_soup = BeautifulSoup(f.read(),'lxml')

    all_text = ""

    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
    return all_text
# Once done, try this:
# get_text("Philosophers/Acrion.html")

# Question 2: Non-Function

In [64]:
# looks at page index.html (all philosophers)
filename = "Philosophers/Acrion.html"

import codecs
from bs4 import BeautifulSoup
f = codecs.open(filename, 'r', 'utf-8')
page_soup = BeautifulSoup(f.read(),'lxml')


all_text = ""

# takes the 2nd index from list philosopher_list
for philosopher_filepath in philosopher_list:
    philosopher_filepath = philosopher_filepath[1]
    
    filepath = codecs.open(philosopher_filepath, 'r', 'utf-8')
    page_soup = BeautifulSoup(filepath.read(),'lxml')
    
    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
# all_text

# Question 3 (functional)

In [81]:
# """
# Question 3

# Use the files under "Philosophers" folder to construct an LSI model.
# Then, use the LSI model to find the most similar philosopher for each of the philosophers
# found in Question 1, based on the content of their Wikipedia articles. You should not go
# online to scrape the data; everything you need is in your Jupyter notebook working directory.

# The function should have as input the list of tuples created in Question 1.

# The output format should be a list of tuples too. Each tuple should contain a philosopher's name
# and its most similar other philosopher. Please note both names can't be the same.

# The output should look like that:

# [('Acrion', 'Arignote'),
#  ('Adrastus of Aphrodisias', 'Lycophron (Sophist)'),
#  ('Aedesia', 'Heliodorus of Alexandria'),
#  ('Aedesius', 'Chrysanthius'),
#  ('Aeneas of Gaza', 'Archytas'),
#  ...]


# """

# def run1(filenames):
# ###
# ### YOUR CODE HERE
# ###

# # Once done, try this:
# run(filenames)

# Question 3 (non-functional)

In [66]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity

In [184]:
# list of all philosophers' file path
philosopher_files = list()
for philosopher_path in filenames:
    philosopher_files.append(philosopher_path[1])

# grab each philosopher's <p> and append to list
# user defined function: get_text()
philosopher_text_list = list()
for philosopher_path in philosopher_files:
    philosopher_text_list.append(get_text(philosopher_path))



In [82]:
# philosopher_text_list[0]

In [89]:
# text preprocessing
# philosopher_text_list = contains multiple indexes of philosophers' description
strip_text_list = list()

for philosopher_text in philosopher_text_list:
    striptext = philosopher_text.replace('\n\n', ' ')
    striptext = striptext.replace('\n', ' ')

##     tokenize all texts: only used when creating LDA
#     sentences = sent_tokenize(striptext)

    strip_text_list.append(striptext)

len(strip_text_list)


92

In [182]:
# lowercase all characters, split the words and remove all special characters
texts = [[word for word in document.lower().split()
         if word not in STOPWORDS and word.isalnum()]
         for document in strip_text_list]

# create word dictionary for each word in a text 
dictionary = corpora.Dictionary(texts)

# create corpus: assigns ints to words
# https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow
# returns (token_id,token_count)
corpus = [dictionary.doc2bow(text) for text in texts]



'Athenodorus of Soli (Greek: Ἀθηνόδωρος ὁ Σολεύς) was a Stoic philosopher, and disciple of Zeno of Citium, who lived in the 3rd century BC. He was the son of Athenodorus, and was born in the town of Soli, Cilicia, and was the compatriot of another disciple of Zeno, Chrysippus. Athenodorus was the brother of the poet Aratus of Soli,[1] the author of the long didactic poem, Phaenomena. Both brothers followed the teachings of Zeno. He is mentioned in the list given by Diogenes Laërtius as the disciple of Zeno.[2] He may be the dedicatee of the work On Definite Propositions (Greek: Περὶ τῶν ϰαταγορευτιϰῶν) written by Chrysippus.[2] '

In [172]:
# train LSI model
lsi = models.LsiModel(corpus, id2word=dictionary,num_topics=2)

In [187]:
# ex) index 1 [Acrion]
# text_acrion_bow = [[word for word in document.lower()
#          if word not in STOPWORDS and word.isalnum()]
#          for document in strip_text_list[0]]

text_acrion_bow = []
acrion_words_list = strip_text_list[0].split()
for word in acrion_words_list:
    if word.isalnum() == True:
        text_acrion_bow.append(word)

# vectorize new bow (bag of words)
acrion_vec_bow = dictionary.doc2bow(text_acrion_bow)
vec_lsi = lsi[acrion_vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
sims = sorted(enumerate(sims),key=lambda item: -item[1])
# acrion is most similar to index 75 = Athenodorus of Soli
sims


[(75, 0.9999115),
 (8, 0.99764264),
 (55, 0.99389684),
 (7, 0.9929013),
 (53, 0.9917134),
 (84, 0.9904734),
 (40, 0.98420525),
 (66, 0.9833431),
 (41, 0.98281384),
 (39, 0.977679),
 (90, 0.9769584),
 (2, 0.9760174),
 (67, 0.9750929),
 (57, 0.9748107),
 (59, 0.97377586),
 (12, 0.9724612),
 (44, 0.97212505),
 (48, 0.9718428),
 (69, 0.9711671),
 (26, 0.97099),
 (24, 0.97088516),
 (43, 0.97019947),
 (9, 0.9684013),
 (42, 0.9676485),
 (76, 0.9669211),
 (86, 0.96555626),
 (3, 0.96446276),
 (88, 0.9590136),
 (21, 0.95733833),
 (45, 0.95649356),
 (38, 0.9563924),
 (47, 0.95436805),
 (11, 0.9535912),
 (65, 0.9527466),
 (37, 0.95259184),
 (36, 0.9521341),
 (74, 0.9508228),
 (83, 0.9477255),
 (50, 0.9465109),
 (78, 0.9464924),
 (85, 0.941419),
 (91, 0.9327191),
 (56, 0.9320204),
 (27, 0.9270043),
 (58, 0.924785),
 (18, 0.9204238),
 (79, 0.91870624),
 (72, 0.918166),
 (14, 0.91288424),
 (51, 0.9128355),
 (22, 0.91213596),
 (34, 0.9112351),
 (73, 0.91067874),
 (6, 0.9105065),
 (29, 0.909552),
 (5, 

In [72]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [73]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [74]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
